In [1]:
import tushare as ts
import pandas as pd

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [3]:
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,fullname,name,area,industry,list_date')

In [4]:
len(data)

3829

In [5]:
data['list_date'] = pd.to_datetime(data['list_date'])

In [6]:
data.head()

,ts_code,name,area,industry,fullname,list_date
0,000001.SZ,平安银行,深圳,银行,平安银行股份有限公司,1991-04-03
1,000002.SZ,万科A,深圳,全国地产,万科企业股份有限公司,1991-01-29
2,000004.SZ,国农科技,深圳,生物制药,深圳中国农大科技股份有限公司,1991-01-14
3,000005.SZ,世纪星源,深圳,环境保护,深圳世纪星源股份有限公司,1990-12-10
4,000006.SZ,深振业A,深圳,区域地产,深圳市振业(集团)股份有限公司,1992-04-27


In [7]:
data.columns = ['companyId:ID(Company)', 'name:string', 'area:string', 'industry:string', 'fullname:string', 'listDate:date']

In [8]:
data.head()

,companyId:ID(Company),name:string,area:string,industry:string,fullname:string,listDate:date
0,000001.SZ,平安银行,深圳,银行,平安银行股份有限公司,1991-04-03
1,000002.SZ,万科A,深圳,全国地产,万科企业股份有限公司,1991-01-29
2,000004.SZ,国农科技,深圳,生物制药,深圳中国农大科技股份有限公司,1991-01-14
3,000005.SZ,世纪星源,深圳,环境保护,深圳世纪星源股份有限公司,1990-12-10
4,000006.SZ,深振业A,深圳,区域地产,深圳市振业(集团)股份有限公司,1992-04-27


In [9]:
data[':LABEL'] = 'Company'

In [10]:
data.head()

,companyId:ID(Company),name:string,area:string,industry:string,fullname:string,listDate:date,:LABEL
0,000001.SZ,平安银行,深圳,银行,平安银行股份有限公司,1991-04-03,Company
1,000002.SZ,万科A,深圳,全国地产,万科企业股份有限公司,1991-01-29,Company
2,000004.SZ,国农科技,深圳,生物制药,深圳中国农大科技股份有限公司,1991-01-14,Company
3,000005.SZ,世纪星源,深圳,环境保护,深圳世纪星源股份有限公司,1990-12-10,Company
4,000006.SZ,深振业A,深圳,区域地产,深圳市振业(集团)股份有限公司,1992-04-27,Company


In [11]:
data.to_csv('import/company.csv', index=False)